In [1]:
# 1. citesti datele din .dat si creezi loaders

import idk
import feature_extractors
import joblib

# baga aici folderu cu fisierele .dat
data_dir = "F://big_dataset"

X_train, y_train, X_val, y_val, X_test, y_test = feature_extractors.read_vectorized_features_2(data_dir)
train_loader, val_loader, test_loader, scaler = idk.prepare_dataloaders(X_train[:1000], y_train[:1000], X_val[:1000], y_val[:1000], X_test[:1000], y_test[:1000], batch_size=128) # aici aproape imi pusca mie :(
joblib.dump(scaler, "scalers/scaler_big.pkl")

print(X_train.shape, X_val.shape, X_test.shape)

(1105702, 2351) (195124, 2351) (433609, 2351)


In [1]:
import torch
print(torch.__version__)

2.7.1+cpu


In [2]:
python3 -m  pipreqs.pipreqs .

SyntaxError: invalid syntax (932506872.py, line 1)

In [2]:
# 2. creezi modelul (asta ar fi primul si daca e necesar mai fac unul)

import torch.nn as nn
import torch.optim as optim

class Model_v1_BIG(nn.Module):
    def __init__(self, input_dim=2351):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 2048),
            nn.ReLU(),
            nn.BatchNorm1d(2048),
            nn.Dropout(0.5),

            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024), 
            nn.Dropout(0.5), 
            
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),  
           
            nn.Linear(1024, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

In [4]:
# 3. test + train

import torch
import torch.nn as nn
import torch.optim as optim
import idk

print("----------Training on Model_v1_BIG----------\n\n")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model_v1_BIG()
print(model)
results = idk.train_model(model, train_loader, val_loader, device, 20, 0.0001, "models/Model_BIG_v1.pth")

idk.plot_training_metrics(
    results["train_losses"],
    results["val_losses"],
    results["train_accuracies"],
    results["val_accuracies"],
    model_name="Model_BIG_v1"
)

model.load_state_dict(torch.load("models/Model_BIG_v1.pth"))
test_results = idk.evaluate_model_on_test(model, test_loader, device)

idk.plot_roc_curve(test_results["labels"], test_results["probs"])
idk.plot_confusion_matrix(test_results["labels"], test_results["probs"])
idk.evaluate_at_fpr_thresholds(test_results["labels"], test_results["probs"], fpr_targets=[0.01, 0.001])

----------Training on Model_v1_BIG----------


Model_v1_BIG(
  (net): Sequential(
    (0): Linear(in_features=2351, out_features=2048, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=2048, out_features=1024, bias=True)
    (5): ReLU()
    (6): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.5, inplace=False)
    (8): Linear(in_features=1024, out_features=1024, bias=True)
    (9): ReLU()
    (10): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Linear(in_features=1024, out_features=1, bias=True)
    (12): Sigmoid()
  )
)
Epoch 1 | Train Loss: 0.7395 | Train Acc: 0.4920 | Val Loss: 1.0890 | Val Acc: 0.0240


KeyboardInterrupt: 